In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.metrics import roc_auc_score as rs
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


In [2]:

ctpath = "C:/Users/Sharon/Desktop/Machine Learning"
# Read CSV into pandas dataframe
tobacco = pd.read_csv('New_Classes.csv')

tobacco = tobacco.dropna()
tobacco['class']=tobacco['class'].astype(int)
tobacco['type'] = tobacco['type'].astype('category')
tobacco['Student Achievement Rating'] = tobacco['Student Achievement Rating'].astype('category')
cat_columns = tobacco.select_dtypes(['category']).columns
print(cat_columns)
tobacco[cat_columns] = tobacco[cat_columns].apply(lambda x: x.cat.codes)
print(tobacco.head())

tobacco_df = tobacco
print(tobacco.shape)
print(type(tobacco))

Index(['Student Achievement Rating', 'type'], dtype='object')
   BoroCT2010  n_report  school_distance  n_school  subway_distance  \
0     5000900         0      2826.764130         0        79.859155   
1     5000900         0      1703.127328         1       924.970954   
2     5000900         0      2929.402252         0      5998.612261   
3     5007400         0      1740.529846         0      2489.816362   
4     5007400         0       630.587028         0       941.382096   

   n_tobacco  Enrollment  Student Achievement Rating  Percent White  \
0          3        1575                           0          0.072   
1          3         427                           0          0.009   
2          3         589                           0          0.019   
3          2         415                           1          0.092   
4          2         307                           2          0.003   

   Percent Black  Percent Hispanic  Percent Asian  % Female  % Poverty    MHI  \
0  

In [34]:

tobacco_df = tobacco_df[['school_distance', 'n_school',
       'subway_distance', 'n_tobacco', 'Enrollment',
       'Student Achievement Rating', 'Percent White', 'Percent Black',
       'Percent Hispanic', 'Percent Asian', '% Female', '% Poverty', 'MHI',
       'Total Population', 'type', 'latitude', 'longitude']]


In [3]:
tobacco = np.asarray(tobacco)
tobacco = tobacco[:,2:20]

X=tobacco[:,0:-1]
y=tobacco[:,-1]
print(X)

[[  2.82676413e+03   0.00000000e+00   7.98591554e+01 ...,   6.00000000e+00
    4.06405610e+01  -7.40822074e+01]
 [  1.70312733e+03   1.00000000e+00   9.24970954e+02 ...,   1.00000000e+00
    4.06419983e+01  -7.40789748e+01]
 [  2.92940225e+03   0.00000000e+00   5.99861226e+03 ...,   6.00000000e+00
    4.06405610e+01  -7.40822074e+01]
 ..., 
 [  2.07573388e+03   0.00000000e+00   1.68049875e+02 ...,   1.00000000e+00
    4.06298670e+01  -7.41100869e+01]
 [  6.68027250e+02   0.00000000e+00   2.08501764e+03 ...,   6.00000000e+00
    4.06291833e+01  -7.41139374e+01]
 [  2.08213877e+03   0.00000000e+00   9.30837987e+02 ...,   6.00000000e+00
    4.06317539e+01  -7.41243431e+01]]


In [37]:
res=[]
X_train,X_test,Y_train,Y_test=train_test_split(X, y, test_size=0.3,random_state=999)
for i in range(1,50,2):
    rf = RFC(n_estimators=i, n_jobs=10,max_leaf_nodes=10)
    rf.fit(X_train, Y_train)
    pred=rf.predict_proba(X_test)[:,1]

    print(1.0*(rf.predict(X_test)==np.asarray(Y_test)).sum()/len(Y_test))

0.835010783609
0.837526959022
0.838245866283
0.838245866283
0.838245866283
0.838245866283
0.838245866283
0.838245866283
0.838245866283
0.838245866283
0.838245866283
0.838245866283
0.838245866283
0.838245866283
0.838245866283
0.838245866283
0.838245866283
0.838245866283
0.838245866283
0.838245866283
0.838245866283
0.838245866283
0.838245866283
0.838245866283
0.838245866283


In [30]:
#for the whole dataset


rf = RFC(n_estimators=8, n_jobs=4,max_leaf_nodes=10)
rf.fit(X_train, Y_train)
correct_IS=1.0*(rf.predict(X_train)==np.asarray(Y_train)).sum()/len(Y_train)
correct_OS=1.0*(rf.predict(X_test)==np.asarray(Y_test)).sum()/len(Y_test)

print("we successfully(IS) predict the {:.4} percent of data".format((correct_IS)*100))
print("we successfully(OS) predict the {:.4} percent of data".format((correct_OS)*100))
precision = precision_score(Y_test, rf.predict(X_test), average="macro")
recall = recall_score(Y_test, rf.predict(X_test), average="macro")  
print("the precision score is ", precision)
print("the recall score is ", recall)

we successfully(IS) predict the 85.04 percent of data
we successfully(OS) predict the 84.67 percent of data
the precision score is  0.211677707595
the recall score is  0.25


/Users/Sharon/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [31]:
print(rf.feature_importances_)

[ 0.08856841  0.02259389  0.05381763  0.10606427  0.03762291  0.01697605
  0.04320433  0.01154457  0.02649625  0.01947673  0.02461378  0.0572894
  0.15907678  0.11528517  0.01091843  0.0956628   0.11078858]


In [33]:
print("Important Features with Random forest model")

Feature_importance=pd.DataFrame([list(tobacco_df.columns),list(rf.feature_importances_)]).T
Feature_importance.columns=["variables","importance"]
important_features_cont = Feature_importance.sort_values(by="importance",ascending=False).iloc[:5,:]
important_features_cont

Important Features with Random forest model


,variables,importance
12,MHI,0.159077
13,Total Population,0.115285
16,longitude,0.110789
3,n_tobacco,0.106064
15,latitude,0.0956628
